In [8]:
import sys
import os
# import calib_eq_odds as eq
cwd = os.getcwd()
print(cwd)
sys.path.append(cwd + "/Post-Processing")
from calib_eq_odds import Model
import pandas as pd
import numpy as np
import torch
import torch.nn as nn


fn_rate = 1
fp_rate = 0

kiva_label = "dataAug/kiva_label.csv"
student_label = "dataAug/student_label.csv" 

/home/li3975/cs587


In [9]:
data = pd.read_csv(kiva_label)
data['repayment_interval'] = data['repayment_interval'].replace(2, 1)

train_data = data [0:20000]
target = train_data['repayment_interval']

target = torch.tensor(target.values).float()
X = train_data.drop(['repayment_interval'], axis=1)
X = torch.tensor(X.values).float()
gender_index = data.columns.get_loc('borrower_genders')

# val_test_data = data[20000:]
# val_testl_target = val_test_data['repayment_interval']
# val_test_target = torch.tensor(val_test_data.values).float()
# val_test_X = val_test_data.drop(['repayment_interval'], axis=1)
# val_test_X = torch.tensor(val_test_X.values).float()
val_data = data[20000: 40000]
val_target = val_data['repayment_interval']
val_target = torch.tensor(val_target.values).float()
val_X = val_data.drop(['repayment_interval'], axis=1)
val_X = torch.tensor(val_X.values).float()
test_data = data[40000:]
test_target = test_data['repayment_interval']
test_target = torch.tensor(test_target.values).float()
test_X = test_data.drop(['repayment_interval'], axis=1)
test_X = torch.tensor(test_X.values).float()


In [10]:
class LogisticRegression(nn.Module):
    def __init__(self):
        # Call the parent constructor to initialize the nn.Module
        super(LogisticRegression, self).__init__()

        # Define a linear transformation layer (weights and bias) for the model.
        # `x_female.shape[1]` specifies the number of input features, and `out_features=1` indicates a single output for binary classification.
        # The bias term is included by setting bias=True, allowing the model to learn an intercept.
        self.w = nn.Linear(9, out_features=1, bias=True)

        # Initialize a sigmoid activation function, which will be used to convert logits to probabilities.
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        # Define the forward pass for the model.
        # Apply the linear transformation to the input `x`.
        w = self.w(x)

        # Apply the sigmoid function to the output of the linear layer.
        # This converts the logits (raw output of the linear layer) to probabilities in the range (0, 1).
        output = self.sigmoid(w)

        # Return the output probabilities.
        return output


In [13]:

model = LogisticRegression()
ceriation = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
model.train()
for epoch in range(50):
    optimizer.zero_grad()
    y_pred = model(X)
    loss = ceriation(y_pred.squeeze(), target)
    # print(f'Epoch {epoch}: y_pred: {y_pred.squeeze()}')
    print(f'Epoch {epoch}: train loss: {loss.item()}')
    loss.backward()
    optimizer.step()


Epoch 0: train loss: 23.014047622680664
Epoch 1: train loss: 16.86544418334961
Epoch 2: train loss: 12.541253089904785
Epoch 3: train loss: 9.297276496887207
Epoch 4: train loss: 6.8834404945373535
Epoch 5: train loss: 5.491730213165283
Epoch 6: train loss: 4.827389717102051
Epoch 7: train loss: 4.43923807144165
Epoch 8: train loss: 4.270114421844482
Epoch 9: train loss: 4.89850378036499
Epoch 10: train loss: 5.0890069007873535
Epoch 11: train loss: 5.279667854309082
Epoch 12: train loss: 5.304134368896484
Epoch 13: train loss: 5.352044105529785
Epoch 14: train loss: 5.3869733810424805
Epoch 15: train loss: 5.428093910217285
Epoch 16: train loss: 5.425224781036377
Epoch 17: train loss: 5.431719779968262
Epoch 18: train loss: 5.4386372566223145
Epoch 19: train loss: 5.437448501586914
Epoch 20: train loss: 5.436474800109863
Epoch 21: train loss: 5.448347091674805
Epoch 22: train loss: 5.448090076446533
Epoch 23: train loss: 5.448210716247559
Epoch 24: train loss: 5.452722549438477
Epoch 

In [14]:
# with no grad
group_0_val_pred = []
group_1_val_pred = []
group_0_val_gt= []
group_1_val_gt = []

group_0_test_pred = []
group_1_test_pred = []
group_0_test_gt= []
group_1_test_gt = []



model.eval()
with torch.no_grad():
    for x, y in zip(val_X, val_target):
        y_pred = model(x)
        if x[gender_index] == 0:
            group_0_val_pred.append(y_pred.item())
            group_0_val_gt.append(y.item())
        else:
            group_1_val_pred.append(y_pred.item())
            group_1_val_gt.append(y.item())
        # print(f'Prediction: {y_pred.item()}, True: {y.item()}')
    
    for x, y in zip(test_X, test_target):
        y_pred = model(x)
        if x[gender_index] == 0:
            group_0_test_pred.append(y_pred.item())
            group_0_test_gt.append(y.item())
        else:
            group_1_test_pred.append(y_pred.item())
            group_1_test_gt.append(y.item())
        # print(f'Prediction: {y_pred.item()}, True: {y.item()}')

In [ ]:
group_0_val_model = Model(np.array(group_0_val_pred), np.array(group_0_val_gt))
group_1_val_model = Model(np.array(group_1_val_pred), np.array(group_1_val_gt))
group_0_test_model = Model(np.array(group_0_test_pred), np.array(group_0_test_gt))
group_1_test_model = Model(np.array(group_1_test_pred), np.array(group_1_test_gt))

In [18]:
_, _, mix_rates = Model.calib_eq_odds(group_0_val_model, group_1_val_model, fp_rate, fn_rate)
calib_eq_odds_group_0_test_model, calib_eq_odds_group_1_test_model = Model.calib_eq_odds(
    group_0_test_model, group_1_test_model, fp_rate, fn_rate, mix_rates)
print('Original group 0 model:\n%s\n' % repr(group_0_test_model))
print('Original group 1 model:\n%s\n' % repr(group_1_test_model))
print('Equalized odds group 0 model:\n%s\n' % repr(calib_eq_odds_group_0_test_model))
print('Equalized odds group 1 model:\n%s\n' % repr(calib_eq_odds_group_1_test_model))

Original group 0 model:
Accuracy:	0.937
F.P. cost:	nan
F.N. cost:	0.050
Base rate:	0.999
Avg. score:	0.949

Original group 1 model:
Accuracy:	0.773
F.P. cost:	nan
F.N. cost:	0.209
Base rate:	0.998
Avg. score:	0.780

Equalized odds group 0 model:
Accuracy:	0.937
F.P. cost:	nan
F.N. cost:	0.050
Base rate:	0.999
Avg. score:	0.949

Equalized odds group 1 model:
Accuracy:	0.773
F.P. cost:	nan
F.N. cost:	0.209
Base rate:	0.998
Avg. score:	0.780



/home/li3975/cs587/Post-Processing/calib_eq_odds.py:82: RuntimeWarning: Mean of empty slice.
  return self.pred[self.label == 0].mean()
/home/li3975/.conda/envs/cent7/2024.02-py311/work/lib/python3.10/site-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
